# Coref resolution

In [1]:
import pickle

# load saved model
with open('allen_coref' , 'rb') as f:
    predictor = pickle.load(f)

#text = """A cat is walking on a sunny day. It jumped over a stone. It died. Dog is walking in the opposite direction. It ran. The dog said "Hello world"."""
text = """Once there lived a crow. He wished to be colorful and beautiful like other birds. 
He then went to the parrot and shared his thoughts. 
But parrot said "Peacock is most beautiful bird so talk to him". 
Then the crow went to the peacock and told him about his looks. 
Then the peacock replied, "You are the luckiest bird that has been never caged in life and we because of our beauty stay caged, and you are always free."
After listening to this, crow realized his mistake and thanked God for making him like this and he flew away happily."""
prediction = predictor.predict(document=text)  # get prediction
print("Clsuters:-")
for cluster in prediction['clusters']:
    print(cluster)  # list of clusters (the indices of spaCy tokens)
# Result: [[[0, 3], [26, 26]], [[34, 34], [50, 50]]]
print('\n\n') #Newline

coref_text = predictor.coref_resolved(text)

print('Coref resolved: ', coref_text)  # resolved text
# Result: Joseph Robinette Biden Jr. is an American politician who is the 46th and

c:\Users\Ujwal\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Clsuters:-
[[3, 4], [6, 6], [18, 18], [26, 26], [47, 48], [67, 67], [89, 89], [101, 101], [103, 103], [110, 110], [114, 114]]
[[22, 23], [31, 31]]
[[34, 34], [42, 42], [51, 52], [55, 55], [57, 57], [62, 63]]
[[80, 80], [83, 83]]



Coref resolved:  Once there lived a crow. a crow wished to be colorful and beautiful like other birds. 
a crow then went to the parrot and shared a crow's thoughts. 
But the parrot said "Peacock is most beautiful bird so talk to Peacock". 
Then a crow went to Peacock and told Peacock about Peacock's looks. 
Then Peacock replied, "a crow are the luckiest bird that has been never caged in life and we because of we's beauty stay caged, and a crow are always free."
After listening to this, a crow realized a crow's mistake and thanked God for making a crow like this and a crow flew away happily.


# SAO parsing

In [2]:
# !pip install textacy

In [7]:
import re
from clausIE import svo

coref_text = """A cat walks rapidly on a sunny day. A cat fell over a stone. A cat died. Dog said "Hello World"."""
SVOs = []
for line in coref_text.split('.'):
    SVO = svo.get_svos(line)
    result = re.search(r"([\"\'])(?:(?=(\\?))\2.)*?\1", line)
    if result:
        SVO = [(str(SVO[0].subject), str(SVO[0].verb), result.group().strip("\""))]
    print(line)
    print(SVO)
    SVOs += (SVO)

print(SVOs)

A cat walks rapidly on a sunny day
[<SV, A cat, walks, None, None, None, [rapidly, on a sunny day]>]
 A cat fell over a stone
[<SVA,  A cat, fell, None, None, None, [over a stone]>]
 A cat died
[<SV,  A cat, died, None, None, None, []>]
 Dog said "Hello World"
[(' Dog', 'said', 'Hello World')]

[]
[<SV, A cat, walks, None, None, None, [rapidly, on a sunny day]>, <SVA,  A cat, fell, None, None, None, [over a stone]>, <SV,  A cat, died, None, None, None, []>, (' Dog', 'said', 'Hello World')]


In [4]:
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))
svos = defaultdict(list)

for svo in SVOs: 
    subject = svo[0]
    word_tokens = word_tokenize(subject)
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    if svo[1] == "said":
        verb = []
        verb.append(svo[1])
        verb.append(svo[2])
    else:
        verb = svo[1]
    svos["".join(filtered_sentence)].append(verb)
    
svos = dict(svos)
print(svos)

charcaters = list(i.lower() for i in svos.keys())
print(charcaters)

IndexError: tuple index out of range

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

actions_movement = {'die' : 0, 'fall' : 0, 'hurt' : 0, 'idle' : 0, 'jump' : 1, 'run' : 1, 'slide' : 1, 'walk' : 1, 'say': 0}
#word2vec similarity between the incoming action vs the ones we have in list of actions and then set a threshold and execute the action based on it

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shour\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
WordNetLemmatizer().lemmatize("said",'v')

'say'

In [ ]:
# !pip install pygame

In [ ]:
from numpy import character
import pygame
import glob
from time import sleep

from gtts import gTTS



language = 'en'

SIZE = WIDTH, HEIGHT = 1280, 720 #the width and height of our screen
FPS = 7 #Frames per second

screen = pygame.display.set_mode(SIZE)
bg = pygame.transform.scale(pygame.image.load('pygame\sunny_day.png') , SIZE)

dir_list = {'r' : 200, 'l' : 400} #moving towards which direction

dialogues = []
dialogue_count = 0
 
class MySprite(pygame.sprite.Sprite):

    def __init__(self, char, x, y, dir):
        super(MySprite, self).__init__()
        
        self.images = {}
        self.actions = []
        # self.images = [pygame.image.load(img) for img in glob.glob("pygame\\cat\\*.png")]
        for action in svos[char]:
            if type(action) == list:
                dialogues.append(action[1])
            action = WordNetLemmatizer().lemmatize(action[0] if type(action) == list else action,'v')   
            self.actions.append(action)
            print(self.actions)
            
            if action not in self.images:
                self.images[action] = [pygame.transform.scale(pygame.image.load(img) , (200,200)) for img in glob.glob("pygame\\" + char + "\\" + action + "*.png")]
            
        print(self.images)
        self.char = char
        self.index = 0
        self.action_count = 0
        self.dir = dir
        self.x = x
        self.y = y   

    def play_dialogue(self, index):
        myobj = gTTS(text=dialogues[index], lang=language, slow=False)
        myobj.save("dialogue.mp3")
        pygame.mixer.init()
        pygame.mixer.music.load("dialogue.mp3")
        pygame.mixer.music.set_volume(1)
        pygame.mixer.music.play()
        
    def movement_update(self):
        # if self.x == 880:
        #     self.dir = 'l'
        # elif self.x == 0:
        #     self.dir = 'r'

        if self.dir == 'r':                        
            self.x += FPS
        else:
            self.image = pygame.transform.flip(self.image, True, False)
            self.x -= FPS
        screen.blit(self.image, (self.x, self.y))
 
    def update(self):
        # print(self.index, self.action_count, len(self.actions), len(self.images[self.actions[self.action_count]]))
        print(self.actions[self.action_count])
        if self.actions[self.action_count] == "say":
            global dialogue_count
            self.play_dialogue(dialogue_count)
            dialogue_count += 1
            print("dialogie played")
        if self.index >= len(self.images[self.actions[self.action_count]]):
            if self.action_count >= len(self.actions) - 1:
                return 1
            self.index = 0
            self.action_count += 1
        self.image = self.images[self.actions[self.action_count]][self.index]
        self.index += 1

        if actions_movement[self.actions[self.action_count]]:
            self.movement_update()
        else:
            screen.blit(self.image, (self.x, self.y))
        
        return 0

    def update_idle(self):
        self.image = pygame.transform.scale(pygame.image.load("pygame\\" + self.char + "\\" + "idle1.png") , (200,200))
        flip_var = True if self.dir == "l" else False
        print(flip_var)
        self.image = pygame.transform.flip(self.image, flip_var, False)
        screen.blit(self.image, (self.x, self.y))
        
 
def main():
    pygame.init()
    pygame.display.set_caption("Trace")
    char_objects = []
    for i in range(len(charcaters)):
        dir = list(dir_list.keys())[i % 2]
        char_objects.append(MySprite(charcaters[i], dir_list[dir] * ((i % 2) + 1), 350, dir))
    # my_group = pygame.sprite.Group(my_sprite)
    idle_char = []
    clock = pygame.time.Clock()
    
    loop = 1
    count = 0 # number of characters whos actions have been completed


    return_val = 0
    while count < len(charcaters): 

        pygame.event.get() 
        
        screen.fill((0,0,0))        
        screen.blit(bg, (0, 0))

        for char in char_objects:
            return_val = char.update()   
            print(return_val) 
            if return_val == 1:
                char_objects.remove(char)
                idle_char.append(char)
                count += return_val
        
        for char in idle_char:
            print(char.x)
            char.update_idle()

        pygame.display.update()
        clock.tick(FPS)

    pygame.quit()
 
if __name__ == '__main__':
    main()

['walk']
['walk', 'jump']
['walk', 'jump', 'die']
{'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>], 'jump': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>], 'die': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>]}
['walk']
['walk', 'run']
['walk', 'run', 'say']
{'walk': [<Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Surface(200x200x32 SW)>, <Sur

# NER

In [ ]:
import spacy 
import spacy.cli
# Download following once
#spacy.cli.download("en_core_web_sm")

In [ ]:
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

In [ ]:
text = "Victoria was a major in CS. Victoria studies at MIT."
doc = nlp(text)

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print(ents)
# now we use displaycy function on doc2
displacy.render(doc, style='ent', jupyter=True)

[('CS', 24, 26, 'ORG'), ('MIT', 48, 51, 'ORG')]


In [ ]:
from gtts import gTTS
  
# This module is imported so that we can 
# play the converted audio
import os
  
# The text that you want to convert to audio
mytext = 'Welcome to geeksforgeeks!'
  
# Language in which you want to convert
language = 'en'
  
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)
  
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save("welcome.mp3")
  
# Playing the converted file
os.system("mpg321 welcome.mp3")

1

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

# object and subject constants
OBJECT_DEPS = {"dobj", "dative", "attr", "oprd"}
SUBJECT_DEPS = {"nsubj", "nsubjpass", "csubj", "agent", "expl"}
# tags that define wether the word is wh-
WH_WORDS = {"WP", "WP$", "WRB"}

# extract the subject, object and verb from the input
def extract_svo(doc):
    sub = []
    at = []
    ve = []
    for token in doc:
        # is this a verb?
        if token.pos_ == "VERB":
            ve.append(token.text)
        # is this the object?
        if token.dep_ in OBJECT_DEPS or token.head.dep_ in OBJECT_DEPS:
            at.append(token.text)
        # is this the subject?
        if token.dep_ in SUBJECT_DEPS or token.head.dep_ in SUBJECT_DEPS:
            sub.append(token.text)
    return " ".join(sub).strip().lower(), " ".join(ve).strip().lower(), " ".join(at).strip().lower()

# wether the doc is a question, as well as the wh-word if any
def is_question(doc):
    # is the first token a verb?
    if len(doc) > 0 and doc[0].pos_ == "VERB":
        return True, ""
    # go over all words
    for token in doc:
        # is it a wh- word?
        if token.tag_ in WH_WORDS:
            return True, token.text.lower()
    return False, ""

# gather the user input and gather the info
doc = nlp("A cat is walking on a sunny day. A cat jumped. A cat was talking with a human and died. Dog is walking in the opposite direction. Dog ran.")
print(doc)
# print out the pos and deps
for token in doc:
    print("Token {} POS: {}, dep: {}".format(token.text, token.pos_, token.dep_))

# get the input information
subject, verb, attribute = extract_svo(doc)
question, wh_word = is_question(doc)
print("svo:, subject: {}, verb: {}, attribute: {}, question: {}, wh_word: {}".format(subject, verb, attribute, question, wh_word))

A cat is walking on a sunny day. A cat jumped. A cat was talking with a human and died. Dog is walking in the opposite direction. Dog ran.
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token is POS: AUX, dep: aux
Token walking POS: VERB, dep: ROOT
Token on POS: ADP, dep: prep
Token a POS: DET, dep: det
Token sunny POS: ADJ, dep: amod
Token day POS: NOUN, dep: pobj
Token . POS: PUNCT, dep: punct
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token jumped POS: VERB, dep: ROOT
Token . POS: PUNCT, dep: punct
Token A POS: DET, dep: det
Token cat POS: NOUN, dep: nsubj
Token was POS: AUX, dep: aux
Token talking POS: VERB, dep: ROOT
Token with POS: ADP, dep: prep
Token a POS: DET, dep: det
Token human POS: NOUN, dep: pobj
Token and POS: CCONJ, dep: cc
Token died POS: VERB, dep: conj
Token . POS: PUNCT, dep: punct
Token Dog POS: PROPN, dep: nsubj
Token is POS: AUX, dep: aux
Token walking POS: VERB, dep: ROOT
Token in POS: ADP, dep: prep
Token the POS: DET, dep: det
Toke